In [1]:
db_config = {
    'user': 'root',
    'password': 'red246@#R^',
    'host': 'localhost',
    'port': 33060,
    'schema': 'chembl_35'
}

In [2]:
import pandas as pd
import mysqlx
from tqdm import tqdm
from decimal import *
from Functions import fetch_admet_for_dataframe

/Users/rsarkar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Creating Lipinski functions list...


100%|██████████| 50/50 [00:00<00:00, 631672.29it/s]




Creating Functional group functions list...


100%|██████████| 102/102 [00:00<00:00, 783551.30it/s]


In [3]:
if __name__ == "__main__":
    db_config = {
        'user': 'root',
        'password': 'red246@#R^',
        'host': 'localhost',
        'port': 33060,
        'schema': 'chembl_35'
    }
    
    df = pd.read_csv("datasets/toxic_chemicals.csv")  # Assumes ChEMBL IDs are in this file
    admet_df = fetch_admet_for_dataframe(df, db_config)
    admet_df.to_csv("admet_data_toxins_updated.csv")

Fetching ADMET Data: 100%|██████████| 16261/16261 [00:35<00:00, 455.97it/s] 


In [4]:
df = pd.read_csv("datasets/drug_structures.csv")
admet_df = fetch_admet_for_dataframe(df, db_config)
admet_df.to_csv("admet_data_drugs_updated.csv")

Fetching ADMET Data: 100%|██████████| 2172/2172 [00:08<00:00, 252.90it/s]


In [ ]:
drugs_admet = pd.read_csv("admet_data_drugs.csv")

drugs_admet.head()

In [ ]:
toxins_admet = pd.read_csv("admet_data_toxins.csv")

toxins_admet.head()

In [ ]:
print(drugs_admet.columns)

In [ ]:
print(toxins_admet.columns)

In [ ]:
toxins_admet_columns = set(toxins_admet.columns)
drugs_admet_columns = set(drugs_admet.columns)

drugs_exclusive = drugs_admet_columns - toxins_admet_columns
print(drugs_exclusive)

In [2]:
import pandas as pd
# Example DataFrame with ChEMBL IDs
df1 = pd.read_csv("datasets/toxic_chemicals.csv")  # Example data
df2 = pd.read_csv("datasets/drug_structures.csv")
# Specific standard_type value to search for

In [ ]:
if __name__ == "__main__":
    # df = pd.read_csv("datasets/toxic_chemicals.csv")  # Assumes ChEMBL IDs are in this file
    admet_df = fetch_admet_for_dataframe(df1)
    admet_df.to_csv("admet_data_toxins_updated.csv", index=False)

In [1]:
import pickle

with open('subsets_dict.pickle', 'rb') as infile:
    subsets_dict = pickle.load(infile)

with open('nodes_data.pickle', 'rb') as infile:
    nodes_data = pickle.load(infile)

In [ ]:
list(subsets_dict.values())[0:5]

In [ ]:
list(nodes_data.keys())[0:5]

In [ ]:
list(nodes_data.values())[0:5]

In [ ]:
len(subsets_dict)

In [ ]:
from Functions import createGraph
from Functions import createSubGraphsList
from Functions import structureCSNMap

graph = createGraph(subsets=subsets_dict, node_data=nodes_data, useMCS=True, attrib_name='molecule_type')
connected_graphs = createSubGraphsList(graph)

In [ ]:
len(connected_graphs)

In [ ]:
ns = [len(n) for n in connected_graphs]

print(ns)

In [10]:
import os

os.mkdir("CSN")

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(connected_graphs))):
    subgraph = connected_graphs[i]
    if len(subgraph)<2:
        continue
    file_name=f"CSN/figure{i}"
    structureCSNMap(subgraph=subgraph, highlighting=True, file_path_name=file_name)

In [42]:
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@imgs" height="42" alt="@imgs" width="42"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@desc</span>
            <span style="font-size: 15px; color: #966;">[$index]</span>
        </div>
        <div>
            <span>@fonts{safe}</span>
        </div>
        <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">($x, $y)</span>
        </div>
    </div>
"""

In [41]:
import networkx as nx
from bokeh.plotting import figure, show, output_file
from bokeh.models import GraphRenderer, StaticLayoutProvider, ColumnDataSource, ImageURL
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw
from PIL import Image
import io
import base64

# Function to process PIL image from RDKit DrawMols function
def show_png(data):
    """Takes in PIL image from RDKit DrawMols function and returns base64-encoded PNG image"""
    bio = io.BytesIO(data)
    img = Image.open(bio)
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    encoded = base64.b64encode(buffer.getvalue()).decode('utf-8')
    return f"data:image/png;base64,{encoded}"

# Example molecules
smiles = ["CCO", "CCN", "CCC", "CCCl"]
molecules = [Chem.MolFromSmiles(s) for s in smiles]

# Generate base64 images using show_png function
image_data = []
for mol in molecules:
    img = Draw.MolToImage(mol, size=(300, 300))  # Generate an image
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")  # Save to buffer
    img = show_png(buffer.getvalue())  # Pass buffer content to show_png
    image_data.append(img)

# Create a NetworkX graph
G = nx.Graph()
G.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 0), (0, 2)])

# Get the node positions in a circular layout
pos = nx.circular_layout(G)
nodes = list(G.nodes())
edges = list(G.edges())

# Convert positions to x, y lists
x = [pos[node][0] for node in nodes]
y = [pos[node][1] for node in nodes]

# Create figure
plot = figure(title="NetworkX Graph with Bokeh", x_range=(-1.2, 1.2), y_range=(-1.2, 1.2), tools="")
plot.axis.visible = False

# Create a graph renderer
graph_renderer = GraphRenderer()
graph_renderer.node_renderer.data_source.add(nodes, 'index')
graph_renderer.edge_renderer.data_source.data = dict(start=[edge[0] for edge in edges],
                                                     end=[edge[1] for edge in edges])

# Set up node positions
layout_provider = StaticLayoutProvider(graph_layout=pos)
graph_renderer.layout_provider = layout_provider

# Add image nodes
source = ColumnDataSource(data=dict(x=x, y=y, image=image_data))
image_glyph = ImageURL(url="image", x="x", y="y", w=0.2, h=0.2, anchor="center")  # Increased image size
plot.add_glyph(source, image_glyph)

# Render graph
plot.renderers.append(graph_renderer)

# Save and show plot
output_file("network_graph_modified.html")
show(plot)

In [15]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Slider, HoverTool, FactorRange
from bokeh.transform import dodge

def create_functional_groups_plot(df):
    """
    Create an interactive side-by-side bar plot from a DataFrame with 'drugs' and 'toxins' columns.
    
    Parameters:
    df (pd.DataFrame): DataFrame with 'drugs' and 'toxins' columns and functional groups as index
    """
    # Sort by drugs count in descending order
    df_sorted = df.sort_values('drugs', ascending=False)
    
    # Create ColumnDataSource
    source = ColumnDataSource(data={
        'functional_groups': df_sorted.index.tolist(),
        'drugs': df_sorted['drugs'].values,
        'toxins': df_sorted['toxins'].values
    })
    
    # Create original source for the slider
    original_source = ColumnDataSource(source.data)
    
    # Create figure with proper x_range
    x_range = FactorRange(factors=df_sorted.index.tolist(), range_padding=0.1)
    p = figure(x_range=x_range,
              height=500, 
              width=900,
              title='Functional Groups Distribution in Drugs vs Toxins',
              toolbar_location='right')
    
    # Create side-by-side bars with minimal spacing
    dodge_amount = 0.15  # Reduced dodge amount
    drugs_bars = p.vbar(x=dodge('functional_groups', -dodge_amount, range=p.x_range),
                       top='drugs',
                       width=0.15,  # Reduced width
                       source=source,
                       color='#2196F3',
                       legend_label='Drugs')
    
    toxins_bars = p.vbar(x=dodge('functional_groups', dodge_amount, range=p.x_range),
                        top='toxins',
                        width=0.15,  # Reduced width
                        source=source,
                        color='#FF5722',
                        legend_label='Toxins')
    
    # Customize the plot
    p.xaxis.axis_label = 'Functional Groups'
    p.yaxis.axis_label = 'Count'
    p.xgrid.grid_line_color = None
    p.xaxis.major_label_orientation = 45
    p.legend.click_policy = 'hide'
    p.legend.location = 'top_right'
    
    # Add hover tool
    hover = HoverTool()
    hover.tooltips = [
        ('Functional Group', '@functional_groups'),
        ('Drugs Count', '@drugs'),
        ('Toxins Count', '@toxins')
    ]
    hover.renderers = [drugs_bars, toxins_bars]
    p.add_tools(hover)
    
    # Create slider
    slider = Slider(start=1,
                   end=len(df),
                   value=len(df),
                   step=1,
                   title='Number of Top Functional Groups')
    
    # Create callback for slider
    callback = CustomJS(args=dict(source=source,
                                original=original_source,
                                slider=slider,
                                p=p), code="""
        // Get the current slider value
        const n = slider.value;
        
        // Get the data
        const data = source.data;
        const original_data = original.data;
        
        // Update the data with top n values
        data['functional_groups'] = original_data['functional_groups'].slice(0, n);
        data['drugs'] = original_data['drugs'].slice(0, n);
        data['toxins'] = original_data['toxins'].slice(0, n);
        
        // Update x-range
        p.x_range.factors = data['functional_groups'];
        
        // Trigger update
        source.change.emit();
    """)
    
    # Connect callback to slider
    slider.js_on_change('value', callback)
    
    # Create layout
    layout = column(slider, p)
    
    return layout

In [ ]:
from bokeh.io import output_notebook

output_notebook()

In [ ]:
# Assuming you have a DataFrame like this:
df = pd.DataFrame({
    'drugs': [100, 80, 60, 40, 20],
    'toxins': [90, 70, 50, 30, 10]},
    index=['Alcohol', 'Amine', 'Ether', 'Ester', 'Ketone'])

# Create and show the plot
plot = create_functional_groups_plot(df)
show(plot)

In [64]:
import pickle
# Example input dictionaries
with open('nodes_data.pickle', 'rb') as infile:
    nodes_data = pickle.load(infile)

with open('subsets_dict.pickle', 'rb') as infile:
    subsets_dict = pickle.load(infile)

In [65]:
list(nodes_data.values())[0:5]

[{'fr_C_O': 1,
  'fr_C_O_noCOO': 1,
  'fr_Al_OH': 1,
  'fr_Ar_OH': 0,
  'fr_methoxy': 0,
  'fr_oxime': 0,
  'fr_ester': 0,
  'fr_Al_COO': 1,
  'fr_Ar_COO': 0,
  'fr_COO': 1,
  'fr_COO2': 1,
  'fr_ketone': 0,
  'fr_ether': 0,
  'fr_phenol': 0,
  'fr_aldehyde': 0,
  'fr_quatN': 0,
  'fr_NH2': 1,
  'fr_NH1': 1,
  'fr_NH0': 0,
  'fr_Ar_N': 0,
  'fr_Ar_NH': 0,
  'fr_aniline': 0,
  'fr_Imine': 0,
  'fr_nitrile': 0,
  'fr_hdrzine': 0,
  'fr_hdrzone': 0,
  'fr_nitroso': 0,
  'fr_N_O': 0,
  'fr_nitro': 0,
  'fr_azo': 0,
  'fr_diazo': 0,
  'fr_azide': 0,
  'fr_amide': 1,
  'fr_priamide': 1,
  'fr_amidine': 0,
  'fr_guanido': 0,
  'fr_Nhpyrrole': 0,
  'fr_imide': 0,
  'fr_isocyan': 0,
  'fr_isothiocyan': 0,
  'fr_thiocyan': 0,
  'fr_halogen': 0,
  'fr_alkyl_halide': 0,
  'fr_sulfide': 0,
  'fr_SH': 0,
  'fr_C_S': 0,
  'fr_sulfone': 0,
  'fr_sulfonamd': 0,
  'fr_prisulfonamd': 0,
  'fr_barbitur': 0,
  'fr_urea': 0,
  'fr_term_acetylene': 0,
  'fr_imidazole': 0,
  'fr_furan': 0,
  'fr_thiophene': 0

In [66]:
list(subsets_dict.values())[0:5]

[{'smi1': 'CC(C)C[C@H](NC(=O)[C@@H]1CSSC[C@@H]2NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@@H](NC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](CC(=O)O)NC(=O)[C@@H](N)CC(N)=O)CSSC[C@H](NC(=O)[C@H](C)NC(=O)[C@H](C(C)C)NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](C(C)C)NC2=O)C(=O)N[C@@H]([C@@H](C)O)C(=O)NCC(=O)N1)C(=O)O',
  'smi2': 'CCOC(=O)[C@H](CCc1ccccc1)N[C@@H](C)C(=O)N1[C@H](C(=O)O)C[C@H]2CCCC[C@@H]21',
  'mol1': <rdkit.Chem.rdchem.Mol at 0x513c60040>,
  'mol2': <rdkit.Chem.rdchem.Mol at 0x513c600e0>,
  'tan_sim': 0.1103448275862069,
  'tan_mcs': 0.122},
 {'smi1': 'CC(C)C[C@H](NC(=O)[C@@H]1CSSC[C@@H]2NC(=O)[C@H](CC(C)C)NC(=O)[C@H](CCC(=O)O)NC(=O)[C@@H](NC(=O)[C@H](CCC(=O)O)NC(=O)[C@H](CC(=O)O)NC(=O)[C@@H](N)CC(N)=O)CSSC[C@H](NC(=O)[C@H](C)NC(=O)[C@H](C(C)C)NC(=O)[C@H](CC(N)=O)NC(=O)[C@H](C(C)C)NC2=O)C(=O)N[C@@H]([C@@H](C)O)C(=O)NCC(=O)N1)C(=O)O',
  'smi2': 'N#Cc1c(N2C[C@@H]3NCCO[C@H]3C2)c(F)cc2c(=O)c(C(=O)O)cn(C3CC3)c12',
  'mol1': <rdkit.Chem.rdchem.Mol at 0x513c601d0>,
  'mol2': <rdkit.Chem.rdchem.Mol at

In [67]:
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@imgs" height="100" alt="@imgs" width="100"
                style="float: left; margin: 0px 15px 15px 0px;"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@id</span>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">@mol_type</span>
        </div>
    </div>
"""

In [142]:
import networkx as nx
from bokeh.io import output_file, show
from bokeh.plotting import figure, from_networkx
from bokeh.models import HoverTool, Range1d, MultiLine, Circle
import matplotlib.pyplot as plt
from tqdm import tqdm

# Creating the graph

def show_png(data):
    """Takes in PIL image from RDKit DrawMols function and returns base64-encoded PNG image"""
    bio = io.BytesIO(data)
    img = Image.open(bio)
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    encoded = base64.b64encode(buffer.getvalue()).decode('utf-8')
    return f"data:image/png;base64,{encoded}"

# Generate base64 images using show_png function
# image_data = []
# for mol in molecules:
#     img = Draw.MolToImage(mol, size=(300, 300))  # Generate an image
#     buffer = io.BytesIO()
#     img.save(buffer, format="PNG")  # Save to buffer
#     img = show_png(buffer.getvalue())  # Pass buffer content to show_png
#     image_data.append(img)

def convertImg(img):
    buffer = io.BytesIO()
    img.save(buffer, format="PNG")
    img = show_png(buffer.getvalue())
    return img

subsets_filtered = {}
for key, value in tqdm(subsets_dict.items(), desc="Filtering subsets: "):
    if value['tan_mcs'] >= 0.68:
        subsets_filtered[key] = value

G = nx.Graph()
for key, value in tqdm(nodes_data.items(), desc="Adding nodes: "):
    smi1= key
    G.add_node(smi1, id=value["CHEMBL"], mol_type=value["molecule_type"])

image_data = {}
for key, value in tqdm(subsets_filtered.items(), desc="Adding edges and drawing molecules: "):
    smi1, smi2 = value['smi1'], value['smi2']
    img1 = Draw.MolToImage(Chem.MolFromSmiles(smi1), size=(300,300))
    img1 = convertImg(img1)
    img2 = Draw.MolToImage(Chem.MolFromSmiles(smi2))
    img2 = convertImg(img2)
    image_data[smi1] = img1
    image_data[smi2] = img2
    G.add_edge(smi1, smi2, weight=value['tan_mcs'])

degrees = dict(nx.degree(G))
nx.set_node_attributes(G, name='degree', values=degrees)
nx.set_node_attributes(G, name='imgs', values=image_data)

graph = None
max_len = 0
for component in list(nx.connected_components(G)):
    if len(component) > max_len:
        max_len = len(component)
        graph = component

Adding edges and drawing molecules: 100%|██████████| 979/979 [00:10<00:00, 95.75it/s] 


In [143]:
graph = list(graph)

In [151]:
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges, LinearColorMapper, ColorBar
from bokeh.palettes import Blues8, Purples8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

subgraph = G.subgraph(graph)

degrees = dict(nx.degree(subgraph))
nx.set_node_attributes(subgraph, name='degree', values=degrees)

number_to_adjust_by = 5
adjusted_node_size = dict([(node, (degree+number_to_adjust_by)/100) for node, degree in nx.degree(subgraph)])
nx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

# Get min and max degree for normalization
degree_values = [data['degree'] for _, data in subgraph.nodes(data=True)]
min_degree, max_degree = min(degree_values), max(degree_values)

# Get min and max edge weights for normalization
edge_weights = [data['weight'] for _, _, data in subgraph.edges(data=True)]
min_weight, max_weight = min(edge_weights)-0.2, max(edge_weights)

edge_cmap = Purples8[::-1]
nodes_color = LinearColorMapper(palette="Blues8", low=min_degree, high=max_degree)
edge_color_mapper = LinearColorMapper(palette=edge_cmap, low=min_weight, high=max_weight)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Chemical Space Network of Drugs and Toxic Molecules'

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html\
network_graph = from_networkx(subgraph, nx.spring_layout, scale=20, center=(0, 0))

# network_graph.edge_renderer.data_source.data['weight'] = edge_weights

#Set node sizes and colors according to node degree (color as spectrum of color palette)
network_graph.node_renderer.glyph = Circle(radius=size_by_this_attribute, fill_color={"field": "degree", "transform": nodes_color})
network_graph.edge_renderer.glyph = MultiLine(line_width=1, line_color={"field": "weight", "transform": edge_color_mapper})
#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(radius=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(radius=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)


#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

color_bar = ColorBar(color_mapper = nodes_color)
color_bar_edges = ColorBar(color_mapper=edge_color_mapper)
plot.add_layout(color_bar, "right")
plot.add_layout(color_bar_edges, "below")
plot.renderers.append(network_graph)

show(plot)
#save(plot, filename=f"{title}.html")

In [139]:
import networkx as nx
from bokeh.plotting import figure, show, from_networkx
from bokeh.io import output_notebook
from bokeh.models import MultiLine, LinearColorMapper, ColorBar
from bokeh.palettes import Viridis256

output_notebook()

# Create a sample graph
G = nx.Graph()

# Add weighted edges
edges = [(1, 2, 0.5), (2, 3, 1.5), (3, 4, 2.0), (4, 5, 0.8), (5, 6, 1.2), (6, 1, 1.8)]
G.add_weighted_edges_from(edges)

# Extract weight values
edge_weights = [data['weight'] for _, _, data in G.edges(data=True)]
min_weight, max_weight = min(edge_weights), max(edge_weights)

# Define color mapper for edges
edge_color_mapper = LinearColorMapper(palette=Viridis256, low=min_weight, high=max_weight)

# Create Bokeh graph renderer
graph_renderer = from_networkx(G, nx.spring_layout(G, seed=42), scale=1, center=(0, 0))
graph_renderer.edge_renderer.data_source.data['weight'] = edge_weights

graph_renderer.edge_renderer.glyph = MultiLine(
    line_width=2,
    line_color={"field": "weight", "transform": edge_color_mapper}
)

# Create Bokeh plot
plot = figure(title="Network Graph with Weighted Edge Coloring", x_range=(-1.5, 1.5), y_range=(-1.5, 1.5), tools="pan,wheel_zoom,box_zoom,reset,save")
plot.renderers.append(graph_renderer)

# Add color bar for edge weights
color_bar = ColorBar(color_mapper=edge_color_mapper, location=(0, 0))
plot.add_layout(color_bar, 'right')

# Show interactive plot
show(plot)


Loading BokehJS ...

In [116]:
min_degree, max_degree

(2, 7)

In [108]:
import networkx as nx
from bokeh.plotting import figure, show, from_networkx
from bokeh.io import output_notebook
from bokeh.models import Circle, MultiLine, LinearColorMapper, ColorBar
from bokeh.transform import linear_cmap
from bokeh.palettes import Purples256, Blues256, Reds256

output_notebook()

# Sample graph
G = nx.Graph()

# Add nodes with attributes
nodes = [
    (1, {'molecule_type': 'drugs', 'degree': 3}),
    (2, {'molecule_type': 'drugs', 'degree': 5}),
    (3, {'molecule_type': 'toxins', 'degree': 2}),
    (4, {'molecule_type': 'toxins', 'degree': 7}),
    (5, {'molecule_type': 'drugs', 'degree': 6}),
    (6, {'molecule_type': 'toxins', 'degree': 4})
]
G.add_nodes_from(nodes)

# Add edges with weights
edges = [(1, 2, 0.5), (2, 3, 1.5), (3, 4, 2.0), (4, 5, 0.8), (5, 6, 1.2), (6, 1, 1.8)]
G.add_weighted_edges_from(edges)

# Get min and max degree for normalization
degree_values = [data['degree'] for _, data in G.nodes(data=True)]
min_degree, max_degree = min(degree_values), max(degree_values)

# Get min and max edge weights for normalization
edge_weights = [data['weight'] for _, _, data in G.edges(data=True)]
min_weight, max_weight = min(edge_weights), max(edge_weights)

# Define colormaps for each molecule type
drug_cmap = Blues256
toxin_cmap = Reds256
edge_cmap = Purples256[::-1]

# Assign colors dynamically based on degree using linear_cmap
node_color_mapper_drugs = LinearColorMapper(palette=drug_cmap, low=min_degree, high=max_degree)
node_color_mapper_toxins = LinearColorMapper(palette=toxin_cmap, low=min_degree, high=max_degree)
edge_color_mapper = LinearColorMapper(palette=edge_cmap, low=min_weight, high=max_weight)

# Store node degrees as attributes for color mapping
for node, data in G.nodes(data=True):
    data['color'] = data['degree']

# Create Bokeh plot
plot = figure(title="Interactive Network Graph", x_range=(-1.5, 1.5), y_range=(-1.5, 1.5), tools="pan,wheel_zoom,box_zoom,reset,save")

# Get node positions
pos = nx.spring_layout(G, seed=42)

# Convert NetworkX graph to Bokeh graph renderer
graph_renderer = from_networkx(G, pos, scale=1, center=(0, 0))

graph_renderer.node_renderer.data_source.data['color'] = [data['degree'] for _, data in G.nodes(data=True)]
graph_renderer.node_renderer.glyph = Circle(radius=0.2, fill_color=linear_cmap('color', drug_cmap + toxin_cmap, min_degree, max_degree))
graph_renderer.edge_renderer.glyph = MultiLine(line_width=2, line_color={"field": "weight", "transform": edge_color_mapper})

# Add graph renderer to the plot
plot.renderers.append(graph_renderer)

# Add color bar for edges
color_bar = ColorBar(color_mapper=edge_color_mapper, location=(0, 0))
plot.add_layout(color_bar, 'right')

# Show interactive plot
show(plot)

Loading BokehJS ...

In [6]:
from bokeh.io import show, save
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, CustomJS
import numpy as np
from PIL import Image
import base64
from io import BytesIO

# List of image file paths
image_files = {
    "MIC": "kde_plots/MIC.png",
    "CL": "kde_plots/CL.png",
    "Activity": "kde_plots/Activity.png",
    "Ki": "kde_plots/Ki.png",
    "Kd": "kde_plots/Kd.png",
    "logP": "kde_plots/logP.png",
    "pKa": "kde_plots/pKa.png",
    "Potency": "kde_plots/Potency.png",
    "PPB": "kde_plots/PPB.png",
    "MR": "kde_plots/MR.png",
    "Solubility": "kde_plots/Solubility.png",
    "LC50": "kde_plots/LC50.png",
    "t_half": "kde_plots/t_half.png",
}

# Function to load an image as a base64 string
def load_image_base64(image_path):
    img = Image.open(image_path).convert("RGBA")
    img = img.resize((400, 400))  # Resize for consistency
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    encoded_image = base64.b64encode(buffer.getvalue()).decode('utf-8')
    return f"data:image/png;base64,{encoded_image}"

# Load images as base64 strings
image_data = {key: load_image_base64(path) for key, path in image_files.items()}

# Create a ColumnDataSource
source = ColumnDataSource(data={"image": [image_data[list(image_files.keys())[0]]]})

# Create a figure for image display
p = figure(x_range=(0, 1), y_range=(0, 1), width=900, height=600)
p.image_url(url='image', x=0, y=1, w=1, h=1, source=source)
p.axis.visible = False  # Hide axes

# Dropdown menu for image selection
select = Select(title="Choose an image:", value=list(image_files.keys())[0], options=list(image_files.keys()))

# CustomJS callback for updating the image
callback = CustomJS(args={"source": source, "image_data": image_data}, code="""
    var data = source.data;
    var selected = cb_obj.value;
    data['image'] = [image_data[selected]];
    source.change.emit();
""")

# Attach callback to dropdown menu
select.js_on_change("value", callback)

# Layout and show
layout = column(select, p)
# show(layout)
save(layout, filename="kde_dropdown.html")

/var/folders/w_/ghym_b9j7wzgy0wvhzp_96440000gp/T/ipykernel_31718/2138124108.py:64: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  save(layout, filename="kde_dropdown.html")
/var/folders/w_/ghym_b9j7wzgy0wvhzp_96440000gp/T/ipykernel_31718/2138124108.py:64: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  save(layout, filename="kde_dropdown.html")


'/Users/rsarkar/Projects/kde_dropdown.html'

In [ ]:
import pandas as pd

admet_data_drugs = pd.read_csv("admet_data_drugs.csv")
admet_data_toxins = pd.read_csv("admet_data_toxins.csv")

admet_data_drugs.head()

In [ ]:
len(admet_data_drugs)

In [ ]:
len(admet_data_toxins)

In [ ]:
nested_dict = {
    col: pd.DataFrame({'drugs': admet_data_drugs[col], 'toxins': admet_data_toxins[col]}).dropna()
    for col in admet_data_drugs if col != 'ChEMBL_ID'
}

print(nested_dict)

In [ ]:
for key, value in nested_dict.items():
    n = len(value)
    print(f"Length of {key} = {n}")

In [31]:
import networkx as nx
from bokeh.io import output_file, show
from bokeh.plotting import figure, from_networkx
from bokeh.models import (HoverTool, ColumnDataSource, ImageURL, GraphRenderer, MultiLine)
from rdkit.Chem import Draw
import base64
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np

# Function to generate image URLs from Mol objects
def mol_to_base64(mol):
    if mol is None:
        return None
    img = Draw.MolToImage(mol, size=(150, 150))
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    encoded = base64.b64encode(buffer.getvalue()).decode()
    return f"data:image/png;base64,{encoded}"

# Function to map tan_mcs values to colors
def map_tan_mcs_to_color(value):
    cmap = plt.get_cmap("coolwarm")
    norm_value = (value - 0.5) / 0.5  # Normalize between 0 and 1 assuming values between 0.5 and 1
    return "#%02x%02x%02x" % tuple(int(c * 255) for c in cmap(norm_value)[:3])

# Example input dictionaries
nodes_data = {
    "CCO": {"CHEMBL": "CHEMBL1", "molecule_type": "drug"},
    "CCN": {"CHEMBL": "CHEMBL2", "molecule_type": "toxin"}
}

subsets_dict = {
    1: {
        "smi1": "CCO", "smi2": "CCN", "mol1": None, "mol2": None,
        "tan_sim": 0.8, "tan_mcs": 0.6
    }
}

# Creating the graph
G = nx.Graph()
node_images = {}
edge_list = []
edge_colors = []

data_edges = {"start": [], "end": [], "color": []}

for key, value in subsets_dict.items():
    smi1, smi2 = value["smi1"], value["smi2"]
    G.add_node(smi1, image=mol_to_base64(value["mol1"]))
    G.add_node(smi2, image=mol_to_base64(value["mol2"]))
    G.add_edge(smi1, smi2, tan_sim=value["tan_sim"], tan_mcs=value["tan_mcs"])
    node_images[smi1] = mol_to_base64(value["mol1"])
    node_images[smi2] = mol_to_base64(value["mol2"])
    color = map_tan_mcs_to_color(value["tan_mcs"])
    data_edges["start"].append(smi1)
    data_edges["end"].append(smi2)
    data_edges["color"].append(color)

# Generate positions
pos = nx.spring_layout(G, scale=1, center=(0, 0))
nodes = list(G.nodes)

# Create data source for nodes
source_nodes = ColumnDataSource(
    data={
        "x": [pos[node][0] for node in nodes],
        "y": [pos[node][1] for node in nodes],
        "image": [node_images[node] for node in nodes],
        "molecule": nodes
    }
)

# Create data source for edges
source_edges = ColumnDataSource(data=data_edges)

# Create plot
plot = figure(title="Molecular Network", x_range=(-1, 1), y_range=(-1, 1), tools="pan,wheel_zoom,reset", active_scroll="wheel_zoom")
network_graph = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))
plot.renderers.append(network_graph)

# Assign edge colors using ColumnDataSource
network_graph.edge_renderer.data_source = source_edges
network_graph.edge_renderer.glyph = MultiLine(line_color="color", line_width=2)

# Add image glyphs for molecules
plot.image_url(url="image", x="x", y="y", w=0.1, h=0.1, source=source_nodes, anchor="center")

# Add hover tool
tooltips = [
    ("Molecule", "@molecule"),
]
hover = HoverTool(tooltips=tooltips)
plot.add_tools(hover)

# Show plot
output_file("network_graph.html")
show(plot)

In [57]:
import networkx as nx
from bokeh.io import output_file, show
from bokeh.plotting import figure, from_networkx
from bokeh.models import (HoverTool, ColumnDataSource, ImageURL, GraphRenderer, MultiLine, ColorBar)
from bokeh.palettes import Viridis256
from bokeh.transform import factor_cmap
from rdkit.Chem import Draw
import base64
from io import BytesIO
import numpy as np
from matplotlib.cm import get_cmap
from matplotlib.colors import Normalize, to_hex

# Function to generate image URLs from Mol objects
def mol_to_base64(mol):
    if mol is None:
        return None
    img = Draw.MolToImage(mol, size=(150, 150))
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    encoded = base64.b64encode(buffer.getvalue()).decode()
    return f"data:image/png;base64,{encoded}"

# Example input dictionaries
nodes_data = {
    "CCO": {"CHEMBL": "CHEMBL1", "molecule_type": "drug"},
    "CCN": {"CHEMBL": "CHEMBL2", "molecule_type": "toxin"}
}

subsets_dict = {
    1: {
        "smi1": "CCO", "smi2": "CCN", "mol1": None, "mol2": None,
        "tan_sim": 0.8, "tan_mcs": 0.6
    }
}

# Creating the graph
G = nx.Graph()
node_images = {}
data_edges = {"start": [], "end": [], "tan_mcs": [], "color": []}

for key, value in subsets_dict.items():
    smi1, smi2 = value["smi1"], value["smi2"]
    G.add_node(smi1, image=mol_to_base64(value["mol1"]))
    G.add_node(smi2, image=mol_to_base64(value["mol2"]))
    G.add_edge(smi1, smi2, tan_sim=value["tan_sim"], tan_mcs=value["tan_mcs"])
    node_images[smi1] = mol_to_base64(value["mol1"])
    node_images[smi2] = mol_to_base64(value["mol2"])
    data_edges["start"].append(smi1)
    data_edges["end"].append(smi2)
    data_edges["tan_mcs"].append(value["tan_mcs"])

# Normalize Tanimoto MCS values for color mapping
min_tan_mcs = min(data_edges["tan_mcs"])
max_tan_mcs = max(data_edges["tan_mcs"])
if min_tan_mcs == max_tan_mcs:
    min_tan_mcs -= 0.01
    max_tan_mcs += 0.01

norm = Normalize(vmin=min_tan_mcs, vmax=max_tan_mcs)
cmap = get_cmap("viridis")
data_edges["color"] = [to_hex(cmap(norm(value))) for value in data_edges["tan_mcs"]]

# Generate positions
pos = nx.spring_layout(G, scale=1, center=(0, 0))
nodes = list(G.nodes)

# Create data source for nodes
source_nodes = ColumnDataSource(
    data={
        "x": [pos[node][0] for node in nodes],
        "y": [pos[node][1] for node in nodes],
        "image": [node_images[node] for node in nodes],
        "molecule": nodes
    }
)

# Create data source for edges
source_edges = ColumnDataSource(data=data_edges)

# Create plot
plot = figure(title="Molecular Network", x_range=(-1, 1), y_range=(-1, 1), tools="pan,wheel_zoom,reset")
network_graph = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))
plot.renderers.append(network_graph)

# Assign edge colors using get_cmap
network_graph.edge_renderer.data_source = source_edges
network_graph.edge_renderer.glyph = MultiLine(line_color="color", line_width=2)

# Add image glyphs for molecules
plot.image_url(url="image", x="x", y="y", w=0.1, h=0.1, source=source_nodes, anchor="center")

# Add hover tool
tooltips = [
    ("Molecule", "@molecule"),
    ("Tanimoto MCS", "@tan_mcs")
]
hover = HoverTool(tooltips=tooltips)
plot.add_tools(hover)

# Show plot
output_file("network_graph.html")
show(plot)

TypeError: 'int' object is not callable